In [1]:
import pandas as pd
from bs4 import BeautifulSoup
import spacy
from natasha import (
    Segmenter,
    MorphVocab,
    NewsEmbedding,
    NewsSyntaxParser,
    NewsNERTagger,
    Doc,
    AddrExtractor
)
import json
import random
from datasets import Dataset
from transformers import AutoTokenizer, AutoModelForTokenClassification
from transformers import TrainingArguments, Trainer
import numpy as np
import pymorphy2
morph = pymorphy2.MorphAnalyzer()

D:\Temp\NER\.venv\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
df1 = pd.read_csv('df_cleaned_small.csv')


In [3]:
df1

Unnamed: 0       http://0-9.ru  \
0             0      http://0009.ru   
1             1      http://001k.ru   
2             2     http://003ms.ru   
3             3     http://003rt.ru   
4             4       http://004.ru   
..          ...                 ...   
995         995   http://1-kovka.ru   
996         996    http://1-krim.ru   
997         997      http://1-kz.ru   
998         998  http://1-legion.ru   
999         999     http://1-lic.ru   

    <html><head>\n        <meta name="viewport" content="width=device-width, initial-scale=1.0">\n        <title>Домен продается. Купить в магазине доменов RU-CENTER</title>\n        <meta name="description" content="Узнать подробнее о домене в магазине доменов RU-CENTER.">\n        <meta name="keywords" content="домен, регистрация доменов, РФ, RU, COM, аукцион доменов, хостинг, почта, освобождающиеся домены">\n        <meta name="application-name" content="nic.ru">\n        <meta name="robots" content="noyaca">\n        <meta name="msapplication-square70x70logo" content="/favicon_70x70.png">\n        <meta name="msapplication-square150x150logo" content="/favicon_150x150.png">\n        <meta name="msapplication-wide310x150logo" content="/favicon_310x150.png">\n        <meta name="msapplication-square310x310logo" content="/favicon_310x310.png">\n        <meta property="og:title" id="title" content="Домен продается. Купить в магазине доменов RU-CENTER">\n        <meta property="og:description" id="description" content="Узнать подробнее о домене в магазине доменов RU-CENTER.">\n        <meta name="twitter:card" content="summary_large_image">\n        <meta name="twitter:site" content="@ru_nic">\n        <meta name="twitter:title" content=" Домен продается. Купить в магазине доменов RU-CENTER ">\n        <meta name="twitter:description" content=" Узнать подробнее о домене в магазине доменов RU-CENTER.">\n        <meta name="twitter:image" content="https://storage.nic.ru/ru/images/jpg/zaglushkapark.jpg">\n        <meta property="og:image" content="https://storage.nic.ru/ru/images/jpg/zaglushkapark.jpg">\n        \n        <link rel="icon" type="image/png" href="https://www.nic.ru/favicon.png">\n        <link rel="icon" type="image/png" sizes="192x192" href="https://www.nic.ru/favicon_192x192.png">\n        <link rel="apple-touch-icon" type="image/png" href="https://www.nic.ru/favicon_192x192.png">\n        <link rel="apple-touch-icon" type="image/png" sizes="152x152" href="https://www.nic.ru/favicon_152x152.png">\n        <link rel="apple-touch-icon" type="image/png" sizes="167x167" href="https://www.nic.ru/favicon_167x167.png">\n        <link rel="apple-touch-icon" type="image/png" sizes="180x180" href="https://www.nic.ru/favicon_180x180.png">\n        <link rel="preload" href="/static/frontend-entry/bundle.7709dbece824ed81e2fa.bundle.js" as="script">\n        <link rel="preconnect" href="https://mc.yandex.ru">\n        <link rel="preconnect" href="https://www.google.ru">\n        <link rel="preconnect" href="https://www.google.com">\n        <link rel="preconnect" href="https://www.google-analytics.com">\n        <link rel="preconnect" href="https://www.googletagmanager.com">\n        <link rel="preconnect" href="https://stats.g.doubleclick.net">\n        <meta name="theme-color" content="#ffffff">\n        <!-- Google Analytics -->\n        <!-- <script>window.user_id = '8739050';</script>-->\n        <!-- <script>window.login = '2709010/NIC-D/adm';</script>-->\n        <!-- <script>-->\n        <!-- !function(e,a,n,i,o,s){e.GoogleAnalyticsObject=i,e.ga=e.ga||function(){(e.ga.q=e.ga.q||[]).push(arguments)},e.ga.l=+new Date,o=a.createElement(n),s=a.getElementsByTagName(n)[0],o.async=1,o.src="//www.google-analytics.com/analytics.js",s.parentNode.insertBefore(o,s)}(window,document,"script","ga"),ga("create","UA-54851853-1","auto"),ga(function(e){var a=e.get("clientId");document.cookie="_ga_cid="+a+"; path=/",ga("set","dimension3",a)}),window.user_id&&(ga("set","&uid",window.us

## Организации ##

In [16]:
def extract_organizations(text):
    organizations = []
    words = text.split()

    for word in words:
        parsed = morph.parse(word)[0]

        # Проверка по тегу:
        if any(tag in parsed.tag for tag in ['Orgn']):
            organizations.append((word, "organization"))

        # Доп:
        elif (parsed.tag.POS == 'NOUN' and
              parsed.tag.case == 'nomn' and
              any(x in word.lower() for x in ['ооо', 'зао', 'ао', 'инк', 'ltd', 'оао'])):
            organizations.append((word, "organization"))

    return list(set(organizations))

In [17]:
def create_orgs_df(df):
    data = []
    for idx, text in zip(df.index, df.iloc[:, -1]):
        for org, org_type in extract_organizations(str(text)):
            data.append((idx, org, org_type))

    return pd.DataFrame(data, columns=['index', 'word', 'type']).drop_duplicates()

df_orgs = create_orgs_df(df1)
df_orgs.to_csv('found_organizations1.csv', index=False)

## Имена ##

In [18]:
nlp = spacy.load("ru_core_news_sm")
nlp.max_length = 5000000
def extract_persons(text):
    doc = nlp(str(text))
    return [ent.text for ent in doc.ents if ent.label_ == "PER"]


In [ ]:
def create_persons_df(df):
    data = []
    for idx, text in zip(df.index, df.iloc[:, -1]):
        persons = extract_persons(str(text))
        data.extend(
            (idx, person, 'name')
            for person in persons
        )

    df_persons = pd.DataFrame(data, columns=['index', 'word', 'type'])
    return df_persons.drop_duplicates()

In [19]:

df_persons = create_persons_df(df1[:500])
df_persons.to_csv('found_persons1.csv', index=False)

df_persons2 = create_persons_df(df1[500:])
df_persons2.to_csv('found_persons1-2.csv', index=False)


MemoryError: Unable to allocate 718. MiB for an array with shape (653556, 288) and data type float32

## Местоположение ##

In [16]:
from natasha import Segmenter, MorphVocab, AddrExtractor, Doc

segmenter = Segmenter()
morph_vocab = MorphVocab()
addr_extractor = AddrExtractor(morph_vocab)  # Для адресов

def extract_address(text):
    doc_natasha = Doc(text)
    doc_natasha.segment(segmenter)

    matches = addr_extractor(doc_natasha)  # Передаем разобранный текст
    return [(match.fact.value, "address") for match in matches]


In [20]:
def create_address_df(df):
    data = []
    for idx, text in zip(df.index, df.iloc[:, -1]):
        data.extend(
            (idx, address, addr_type)
            for address, addr_type in extract_address(str(text))
        )

    df_address = pd.DataFrame(data, columns=['index', 'word', 'type'])
    return df_address.drop_duplicates()



In [25]:
df11 = df1[:50]
df12 = df1[50:100]
df13 = df1[100:200]
df14 = df1[300:400]

In [22]:
df_address = create_address_df(df11)
df_address.to_csv('found_address11.csv', index=False)

In [24]:
df_address2 = create_address_df(df12)
df_address2.to_csv('found_address12.csv', index=False)

## Должность ##

In [46]:
nlp = spacy.load("ru_core_news_sm")
nlp.max_length = 5000000
def extract_positions(text):
    try:
        doc = nlp(str(text))
        positions = []

        for token in doc:
            # Выброс
            if token.pos_ != "NOUN" or token.morph.get("Case") != ["Nom"]:
                continue

            # 1: Управление глаголом назначения
            if any(child.lemma_ in {"назначить", "избрать", "утвердить"}
                   for child in token.head.children):
                positions.append((token.text, "position1"))
                continue

            #2: Аппозитивная конструкция с PER/ORG
            if any(child.dep_ == "appos" and child.ent_type_ in {"PER", "ORG"}
                   for child in token.children):
                positions.append((token.text, "position2"))
                continue

            # 3: Составное сказуемое с должностью
            if token.dep_ in {"nsubj", "attr"} and token.head.pos_ == "VERB":
                if token.head.lemma_ in {"работать"}:
                    positions.append((token.text, "position3"))

            #4: Конструкция "в должности X"
            if "PrepCase=Nom" in token.morph and any(
                child.dep_ == "case" and child.text.lower() in {"в", "на"}
                for child in token.children
            ):
                positions.append((token.text, "position4"))

        return list(set(positions))
    except:
        print('error')

In [47]:
def create_positions_df(df):
    data = []
    for idx, text in zip(df.index, df.iloc[:, -1]):
        data.extend(
            (idx, position, pos_type)
            for position, pos_type in extract_positions(text)
        )

    df_positions = pd.DataFrame(data, columns=['index', 'word', 'type'])
    return df_positions.drop_duplicates()


In [48]:
df_positions = create_positions_df(df11)
df_positions.to_csv('found_positions11.csv', index=False)

## Ссылки ##

In [72]:
def generate_url():
    """генерация url для примеров"""
    schemes = ["http://", "https://", "www."]
    domains = ["example", "google", "yandex", "github", "amazon", "mail", "youtube", "cite", "gmail"]
    tlds = [".com", ".ru", ".org", ".net", ".html"]
    paths = ["", "/path", "/page", "/index.html", "/search?q=query"]
    return f"{random.choice(schemes)}{random.choice(domains)}{random.choice(tlds)}{random.choice(paths)}"

def generate_dataset(num_samples=100):
    """генерация примеров самих"""
    data = []
    for _ in range(num_samples):
        url = generate_url()
        templates = [
            f"Посетите {url} для подробностей",
            f"Сайт {url}",
            f"Ссылка: {url} доступна всем",
            f"Перейдите по {url} чтобы узнать больше",
            f"Наш ресурс {url} содержит информацию",
            f"Для поиска информации перейдите {url}",
            f"Вся информация на {url}"
        ]
        text = random.choice(templates)
        start = text.find(url)
        end = start + len(url)
        data.append({
            "text": text,
            "entities": [{"start": start, "end": end, "label": "URL"}]
        })
    return data

dataset = Dataset.from_list(generate_dataset(100))
dataset.save_to_disk("url_dataset")

Saving the dataset (1/1 shards): 100%|██████████| 100/100 [00:00<00:00, 6233.45 examples/s]


In [74]:
# Загружаем токенизатор и модель
model_checkpoint = "bert-base-multilingual-cased"
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)
model = AutoModelForTokenClassification.from_pretrained(model_checkpoint, num_labels=2)

# Функция для выравнивания меток
def tokenize_and_align_labels(examples):
    tokenized_inputs = tokenizer(
        examples["text"],
        truncation=True,
        padding="max_length",
        max_length=64,
        return_offsets_mapping=True
    )

    labels = []
    for i, (offsets, entities) in enumerate(zip(tokenized_inputs["offset_mapping"], examples["entities"])):
        label = np.zeros(len(offsets), dtype=int)

        for entity in entities:
            # Помечаем токены, которые полностью входят в URL
            for token_idx, (start, end) in enumerate(offsets):
                if start >= entity["start"] and end <= entity["end"]:
                    label[token_idx] = 1  # 1 - метка для URL

        labels.append(label.tolist())

    tokenized_inputs["labels"] = labels
    return tokenized_inputs

# токенизируем
tokenized_dataset = dataset.map(tokenize_and_align_labels, batched=True)

# Параметры обучения
training_args = TrainingArguments(
    output_dir="url_ner_model",
    per_device_train_batch_size=2,
    num_train_epochs=2,
    learning_rate=2e-5,
    weight_decay=0.01,
    save_strategy="epoch",
    logging_dir="./logs",
    evaluation_strategy="no",
    max_steps=100,
    logging_steps=10,
    save_steps=50,

)

# тренер создание
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset,
    tokenizer=tokenizer,
)

trainer.train()

trainer.save_model("best_url_ner_model")
tokenizer.save_pretrained("best_url_ner_model")

Some weights of BertForTokenClassification were not initialized from the model checkpoint at bert-base-multilingual-cased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Map: 100%|██████████| 100/100 [00:00<00:00, 480.57 examples/s]
D:\Temp\NER\.venv\lib\site-packages\transformers\training_args.py:1611: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
C:\Users\Александра\AppData\Local\Temp\ipykernel_4220\3995399208.py:51: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


Step,Training Loss
10,0.300300
20,0.021900
30,0.000900
40,0.000300
50,0.000200
60,0.000200
70,0.000100
80,0.000100
90,0.000100
100,0.000100


('best_url_ner_model\\tokenizer_config.json',
 'best_url_ner_model\\special_tokens_map.json',
 'best_url_ner_model\\vocab.txt',
 'best_url_ner_model\\added_tokens.json',
 'best_url_ner_model\\tokenizer.json')

In [9]:
from transformers import pipeline, AutoTokenizer, AutoModelForTokenClassification

class URLExtractor:
    def __init__(self, model_path="best_url_ner_model"):
        # Загружаем модель и токенизатор из локальной
        self.tokenizer = AutoTokenizer.from_pretrained(model_path)
        self.model = AutoModelForTokenClassification.from_pretrained(model_path)

        # Создаем pipeline
        self.ner_pipeline = pipeline(
            "token-classification",
            model=self.model,
            tokenizer=self.tokenizer,
            aggregation_strategy="simple"  #  смежные токены --  группировка
        )

    def extract_urls(self, text):
        """извлечение URL из текста с постобработкой"""
        entities = self.ner_pipeline(text)

        # Фильтруем только URL (LABEL_1) и очищаем результаты
        urls = []
        for entity in entities:
            if entity["entity_group"] == "LABEL_1":
                url = entity["word"]

                # лишние пробелы
                url = url.replace(" ", "")

                # валидность (доп)
                if "://" in url or url.startswith(("http", "www", "ftp")):
                    urls.append(url)

        return urls

try:
    extractor = URLExtractor()
    print("Модель успешно загружена!")
except Exception as e:
    print(f"Ошибка загрузки модели: {e}")



Device set to use cpu


Модель успешно загружена!


In [10]:
df = pd.read_csv('analyzer_analyzer_urls.csv')
df

http://0-9.ru  \
0         http://0009.ru   
1         http://001k.ru   
2        http://003ms.ru   
3        http://003rt.ru   
4          http://004.ru   
...                  ...   
9994    http://1hobby.ru   
9995      http://1hod.ru   
9996      http://1hop.ru   
9997  http://1hostels.ru   
9998   http://1hotel1.ru   

     <html><head>\n        <meta name="viewport" content="width=device-width, initial-scale=1.0">\n        <title>Домен продается. Купить в магазине доменов RU-CENTER</title>\n        <meta name="description" content="Узнать подробнее о домене в магазине доменов RU-CENTER.">\n        <meta name="keywords" content="домен, регистрация доменов, РФ, RU, COM, аукцион доменов, хостинг, почта, освобождающиеся домены">\n        <meta name="application-name" content="nic.ru">\n        <meta name="robots" content="noyaca">\n        <meta name="msapplication-square70x70logo" content="/favicon_70x70.png">\n        <meta name="msapplication-square150x150logo" content="/favicon_150x150.png">\n        <meta name="msapplication-wide310x150logo" content="/favicon_310x150.png">\n        <meta name="msapplication-square310x310logo" content="/favicon_310x310.png">\n        <meta property="og:title" id="title" content="Домен продается. Купить в магазине доменов RU-CENTER">\n        <meta property="og:description" id="description" content="Узнать подробнее о домене в магазине доменов RU-CENTER.">\n        <meta name="twitter:card" content="summary_large_image">\n        <meta name="twitter:site" content="@ru_nic">\n        <meta name="twitter:title" content=" Домен продается. Купить в магазине доменов RU-CENTER ">\n        <meta name="twitter:description" content=" Узнать подробнее о домене в магазине доменов RU-CENTER.">\n        <meta name="twitter:image" content="https://storage.nic.ru/ru/images/jpg/zaglushkapark.jpg">\n        <meta property="og:image" content="https://storage.nic.ru/ru/images/jpg/zaglushkapark.jpg">\n        \n        <link rel="icon" type="image/png" href="https://www.nic.ru/favicon.png">\n        <link rel="icon" type="image/png" sizes="192x192" href="https://www.nic.ru/favicon_192x192.png">\n        <link rel="apple-touch-icon" type="image/png" href="https://www.nic.ru/favicon_192x192.png">\n        <link rel="apple-touch-icon" type="image/png" sizes="152x152" href="https://www.nic.ru/favicon_152x152.png">\n        <link rel="apple-touch-icon" type="image/png" sizes="167x167" href="https://www.nic.ru/favicon_167x167.png">\n        <link rel="apple-touch-icon" type="image/png" sizes="180x180" href="https://www.nic.ru/favicon_180x180.png">\n        <link rel="preload" href="/static/frontend-entry/bundle.7709dbece824ed81e2fa.bundle.js" as="script">\n        <link rel="preconnect" href="https://mc.yandex.ru">\n        <link rel="preconnect" href="https://www.google.ru">\n        <link rel="preconnect" href="https://www.google.com">\n        <link rel="preconnect" href="https://www.google-analytics.com">\n        <link rel="preconnect" href="https://www.googletagmanager.com">\n        <link rel="preconnect" href="https://stats.g.doubleclick.net">\n        <meta name="theme-color" content="#ffffff">\n        <!-- Google Analytics -->\n        <!-- <script>window.user_id = '8739050';</script>-->\n        <!-- <script>window.login = '2709010/NIC-D/adm';</script>-->\n        <!-- <script>-->\n        <!-- !function(e,a,n,i,o,s){e.GoogleAnalyticsObject=i,e.ga=e.ga||function(){(e.ga.q=e.ga.q||[]).push(arguments)},e.ga.l=+new Date,o=a.createElement(n),s=a.getElementsByTagName(n)[0],o.async=1,o.src="//www.google-analytics.com/analytics.js",s.parentNode.insertBefore(o,s)}(window,document,"script","ga"),ga("create","UA-54851853-1","auto"),ga(function(e){var a=e.get("clientId");document.cookie="_ga_cid="+a+"; path=/",ga("set","dimension3",a)}),window.user_id&&(ga("set","&uid",window.user_id),ga("set","dimension4",window.user_id)),ga("set","dimension2",window.login?"User":"Guest"),window.send_ab_split_ga_events&&(window.

In [11]:
import re
# очистка для ссылок (почти такая же, как в cleansing, только англ текст не убираем)
def clean_html_content(html_text):
    if pd.isna(html_text) or not str(html_text).strip():
        return ""

    text = str(html_text)
    text = re.sub(r'<style[^>]*>.*?</style>', '', text, flags=re.DOTALL)
    text = re.sub(r'<script[^>]*>.*?</script>', '', text, flags=re.DOTALL)


    try:
        soup = BeautifulSoup(text, 'html.parser')

        for element in soup(['header', 'footer', 'nav', 'iframe', 'noscript', 'form']):
            element.decompose()

        # сохранеение структуру
        text = '\n'.join(element.get_text(separator=' ', strip=True)
                        for element in soup.find_all(True) if element.name not in ['style', 'script'])
    except:
        # простая обработка - ошибка парсера
        text = re.sub(r'<[^>]+>', ' ', text)
    return text

In [12]:
df_small_with_eng = df[:1000]
df_small_with_eng.iloc[:, -1] = df_small_with_eng.iloc[:, -1].apply(clean_html_content)

df_small_with_eng.iloc[:, -1] = df_small_with_eng.iloc[:, -1].str.lower()

In [15]:
def create_url_df(df):
    data = []

    for idx, text in zip(df.index, df.iloc[:, -1]):
        extracted_urls = extractor.extract_urls(text)

        if not isinstance(extracted_urls, (list, tuple)):
            print(f"1Ошибка:  Значение: {extracted_urls}1")
            continue

        for item in extracted_urls:
            print(f"item = {item}")
            url = item
            data.append((idx, url, 'url'))

    df_url = pd.DataFrame(data, columns=['index', 'word', 'type'])
    return df_url



df_url = create_url_df(df_small_with_eng)
df_url.to_csv('found_urls11.csv', index=False)

item = https://vk.com/nrarussia+74953803067
item = храмаик-2уфсинроссиипореспубликеадыгея.https://rutube.ru/video/fb6af92c0d5d7c76eeee3fd8f29a7a22/29.09.2023
item = храмаик-2уфсинроссиипореспубликеадыгея.https://rutube.ru/video/fb6af92c0d5d7c76eeee3fd8f29a7a22/29.09.2023
item = https://vk.com/nrarussia+74953803067
item = https://www.murzilkintravel.ru/2023/09/hristoforich.html
item = thecachefromyourbrowsersettings1.type”opera://settings/clearbrowserdata"intheaddressbar
item = http(web)
item = д.4.п.1http://ds26.kaluga.ru/?section_id=20
item = .https://bus.gov.ru/qrcode/rate/334274
item = http://0302.kmr.msudrf.ru/modules.php?name=info_pages&rid=1022#content
item = httperror403nullreloadyoudon'thaveauthorizationtoviewthispage
item = httperror403nullhttperror403nullhttperror403nullhttperror403nullhttperror403nullnullreloadreloadreloadyoudon'thaveauthorizationtoviewthispage.youdon'thaveauthorizationtoviewthispage
item = httperror403null03-bux.ru03-bux.ruaccessto03-bux.ruwasdeniedyoudon't

## Конкатенация ##

In [1]:
import pandas as pd

files = ["found_address11.csv", "found_address12.csv"]

dfs = []

for file in files:
    df = pd.read_csv(file)
    dfs.append(df)

result = pd.concat(dfs, ignore_index=True)

result.to_csv("found_address1.csv", index=False)


In [3]:
import pandas as pd

files = ["found_address1.csv","found_persons1.csv", "found_organizations1.csv", "found_positions11.csv", "found_urls11.csv"]

dfs = []

for file in files:
    df = pd.read_csv(file)
    dfs.append(df)

result = pd.concat(dfs, ignore_index=True)

result.to_csv("result.csv", index=False)